In [1]:
import os
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.schema import Document, SystemMessage, HumanMessage

In [2]:
load_dotenv()
api_key = os.getenv('GROQ_API_KEY')

In [3]:
chat = ChatGroq(temperature=0, groq_api_key=api_key, model_name="llama3-70b-8192")

In [4]:
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-MiniLM-L6-v2')

c:\Users\andyu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
comet_ml is installed but `COMET_API_KEY` is not set.


In [5]:
persist_directory = "../RAG_3_vectordb_3_separate_codes/paragraphs_chroma_db"

Checking if the directory exists

In [6]:
if not os.path.exists(persist_directory):
    print("Persist directory does not exist.")
else:
    print("Persist directory exists.")

Persist directory exists.


Loading already created vector database. By specifying the embedding function, we are ensuring that the same model used to create the database is being used to query it.

In [7]:
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

Providing additional information about the metadata for more precise filtering and details about the document description helps the chain understand the document's content.

In [8]:
metadata_field_info = [
    AttributeInfo(
        name="year",
        description="Year the paper was published",
        type="integer",
    ),
    AttributeInfo(
        name="title",
        description="Title of the paper",
        type="string",
    ),
    AttributeInfo(
        name="last_section_title",
        description="Title section is associated with paragraph",
        type="string",
    )
]

document_content_description = "Provides context of each paragraph within the article"

Setting up the SelfQueryRetriever that is used for retriving documents from the vector database. In this chain we specify that we are using Llama 3 as our LLM model, which sees analyzes the prompt and structures it into a query, paragraphs_chroma_db as our vector database and provide also the additional attribute info about metadata and document description.

In [9]:
retriever = SelfQueryRetriever.from_llm(
    llm=chat,
    vectorstore=vectorstore,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    verbose=True
)

Creating a custom prompt instructs the model to always respond in full sentences and to say "I don't know" if it doesn't know the answer. This approach prevents Llama 3 from generating random responses simply to fulfill the expectation of an answer.

In [10]:
custom_prompt_template = """Use the following pieces of information to answer the user's question. Always answear the question as if you were a human and answear in full sentance. During your answear be really specific. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [11]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

prompt = set_custom_prompt()

Here, we define the complete question-answering chain in RAG. We specify that Llama 3 is the LLM model in use, the retriever is the SelfQueryRetriever we created earlier, chain_type="stuff" indicates a basic chain where the LLM answers based solely on the user's question and retrieved documents, and finally, we ensure that the custom prompt we created is used.

In [12]:
qa = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={'prompt': prompt}
)

In [13]:
query = "How was study conduted in article: A Multilevel Model of Resistance to Information Technology Implementation, be very specififc"
result = qa({"query": query})
print("Answer:", result["result"])

c:\Users\andyu\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Answer: The study was conducted in two stages: within-case analysis and cross-case analysis. In the within-case analysis, data was analyzed to allow the unique patterns of each case to emerge and to provide researchers with a rich understanding of each case. This was done to accelerate cross-case comparisons. In the cross-case analysis, analytic induction was used to search for common patterns and unique features. The researchers developed hypotheses prior to data analysis, which were revised to fit emerging interpretations of the data over the course of data collection and analysis. To reach an appropriate degree of internal validity, three sources of evidence were used: direct observation, documentation, and interviews. The researchers spent several days at each site observing how each EMR was used, analyzed system and project documentation, and conducted interviews. Data gathered from these sources was used to corroborate, validate, and complement the interview data. Cross-case anal

In [14]:
query = "Is case study mentioned in article When Does Technology Use Enable Network Change in Organizations? A Comparative Study of Feature Use and Shared Affordances"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: Yes, the case study mentioned in the article is about Autoworks, a company in the automotive industry, specifically focusing on the Safety division, where the researcher spent nine months conducting ethnographic data collection and administered sociometric surveys to engineers in the Piston and Strut groups to study the impact of CrashLab technology on their network and advice-seeking behaviors.


In [15]:
query = "Is case study mentioned in article: Understanding User Revisions When Using Information System Features: Adaptive System Use and Triggers. if it is not mentioned, which kind of study was used"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: No, a case study is not mentioned in the article "Understanding User Revisions When Using Information System Features: Adaptive System Use and Triggers". Instead, the study used a survey and Q-sorting method, as described in the Research Design and Step 2: Interviews sections, respectively.


In [17]:
query = "Which groups of people were involved in the survey in article: How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The group of people involved in the survey in the article "How Habit Limits the Predictive Power of Intention: The Case of Information Systems Continuance" were business students at a university in Hong Kong.
